In [7]:
#======================= import essential libraries===========================
import os
import math
import numpy as np
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import matplotlib.tri as mtri
from python_vtu import vtu_extract_element_connectivity,vtu_extract_fields

# Input parameters
Hght = 500               # Height(m)
Lngth = 2e4              # Lngth(m)               
tol = 1e-3               # Tolerance(m)
T = 0.5*3.154e7/(12.0*30.0)  # solar tide period
sea_level = Hght*0.917   # sea level(m)
fs = 14                  # FontSize
marker = 6               # MarkerType
r_lake = 0.5e3           # Lake redius
KIc = 1e5                # Ice fracture toughness

rho_w = 1e3              # water density (kg/m^3)
rho_i = 917              # ice density (kg/m^3)
g = 9.8                  # gravity (m/s^2)
pi = math.pi               # pi
ny = 1e4                   # y grid resolution

basin_elev = 80.0        # basin_elevation (m)
depth_basin = 90.0 - basin_elev  # basin depth (m)

# casename 
vfile = ['../results/stokes_tidal_response_U09ma_L20000_Slope0_02_A3e_24_n3_0_mu0_30e9_deltap1e_13_deltav1e_18_tide1_00_C1_0e7_DX12']

for file in vfile:
    # ========== check if the txt file has a proper subfix ===========
    path = file + "/line_plot_data"
    listName = os.listdir(path)
    for files in listName:
        if files[-4:]=='.txt':
            continue
        else:
            txtName = files + '.txt'
            os.rename(os.path.join(path,files), os.path.join(path,txtName))

    #======================= read the data ===========================
    X = np.loadtxt(file+ "/line_plot_data/X.txt", dtype='f', delimiter=' ')
    Gamma_h = np.loadtxt(file+"/line_plot_data/Gamma_h.txt", dtype='f', delimiter=' ')
    Gamma_s = np.loadtxt(file+ "/line_plot_data/Gamma_s.txt", dtype='f', delimiter=' ')
    x_left = np.loadtxt(file+ "/line_plot_data/x_left.txt", dtype='f', delimiter=' ')
    x_right = np.loadtxt(file+ "/line_plot_data/x_right.txt", dtype='f', delimiter=' ')
    t = np.loadtxt(file+ "/line_plot_data/t.txt", dtype='f', delimiter=' ')

    max_tension = []          # low-tide deviatoric tensile stress
    max_tension_net = []      # low-tide tensile stress
    max_tension_t = []        # time when max tension occurs
    max_tension_x = []        # x-coord at which max tension occurs
    min_tension = []          
    min_tension_t = []
    min_tension_x = []

    v_frame = np.arange(0,np.shape(t)[0],1)
    for n_frame in v_frame:
        print(n_frame)
        xi = X[::] # Grid used to interpolate the stress
        yi = Gamma_h[::,n_frame]-0.05
        xg_bar = x_left.mean()
        lake_range = (xi>=xg_bar-r_lake)&(xi<=xg_bar+r_lake)

        # stress
        n_frame_str="{:0>6d}".format(n_frame)
        filename = file+"/field_plot_data"
        e2v = vtu_extract_element_connectivity(filename+"/sigma"+n_frame_str+".vtu")
        pf, cf, coor = vtu_extract_fields(filename+"/sigma"+n_frame_str+".vtu")
        pf_names = list(pf.keys())
        cf_names = list(cf.keys())
        tri = mtri.Triangulation(coor['coor'][:, 0], coor['coor'][:, 1], e2v) # trangulation
        interp_cubic_geom = mtri.CubicTriInterpolator(tri, pf[pf_names[0]][:,0], kind='geom')
        sigma_xx_c = interp_cubic_geom(xi[lake_range], yi[lake_range])

        # p Interpretor
        filename = file+"/field_plot_data"
        e2v = vtu_extract_element_connectivity(filename+"/p"+n_frame_str+".vtu")
        pf, cf, coor = vtu_extract_fields(filename+"/p"+n_frame_str+".vtu")
        pf_names = list(pf.keys())
        cf_names = list(cf.keys())
        tri = mtri.Triangulation(coor['coor'][:, 0], coor['coor'][:, 1], e2v) # trangulation
        interp_cubic_geom = mtri.CubicTriInterpolator(tri, pf[pf_names[0]], kind='geom')
        p_c = interp_cubic_geom(xi[lake_range], yi[lake_range])
        
        sigma_xx_net = sigma_xx_c - p_c
        
        max_tension_t.append(t[n_frame]/T)
        max_tension.append(np.amax(sigma_xx_c))
        max_tension_net.append(np.amax(sigma_xx_net))
        max_tension_x.append(xi[lake_range][np.argmax(sigma_xx_c)])
        
        min_tension_t.append(t[n_frame]/T)
        min_tension.append(np.amin(sigma_xx_c))
        min_tension_x.append(xi[lake_range][np.argmin(sigma_xx_c)])

    # convert the lists to arrays, then save
    max_tension_t = np.array(max_tension_t)
    max_tension = np.array(max_tension)
    max_tension_net = np.array(max_tension_net)
    
    max_tension_x = np.array(max_tension_x)
    min_tension_t = np.array(min_tension_t)
    min_tension = np.array(min_tension)
    min_tension_x = np.array(min_tension_x)
    
    np.savetxt( file +'/max_tension_t.txt', max_tension_t, delimiter=',')
    np.savetxt( file +'/max_tension.txt', max_tension, delimiter=',')  
    np.savetxt( file +'/max_tension_net.txt', max_tension_net, delimiter=',')  
    np.savetxt( file +'/max_tension_x.txt', max_tension_x, delimiter=',')   
    np.savetxt( file +'/min_tension_t.txt', min_tension_t, delimiter=',')   
    np.savetxt( file +'/min_tension.txt', min_tension, delimiter=',')   
    np.savetxt( file +'/min_tension_x.txt', min_tension_x, delimiter=',')   

    print('Case '+file+' finished!')


0
#point fields 1
  pointFields.name stress
#cell fields 0
#point fields 1
  pointFields.name press
#cell fields 0
1
#point fields 1
  pointFields.name stress
#cell fields 0
#point fields 1
  pointFields.name press
#cell fields 0
2
#point fields 1
  pointFields.name stress
#cell fields 0
#point fields 1
  pointFields.name press
#cell fields 0
3
#point fields 1
  pointFields.name stress
#cell fields 0
#point fields 1
  pointFields.name press
#cell fields 0
4
#point fields 1
  pointFields.name stress
#cell fields 0
#point fields 1
  pointFields.name press
#cell fields 0
5
#point fields 1
  pointFields.name stress
#cell fields 0
#point fields 1
  pointFields.name press
#cell fields 0
6
#point fields 1
  pointFields.name stress
#cell fields 0
#point fields 1
  pointFields.name press
#cell fields 0
7
#point fields 1
  pointFields.name stress
#cell fields 0
#point fields 1
  pointFields.name press
#cell fields 0
8
#point fields 1
  pointFields.name stress
#cell fields 0
#point fields 1
  poi